In [2]:
import pandas as pd
import random
from sklearn.metrics import roc_auc_score

In [19]:
df = pd.read_csv("./kaggle-dataset/train_features.csv", index_col='id')
out = pd.read_csv("./kaggle-dataset/train_labels.csv", index_col='id')
df["charged_off"] = out["charged_off"].astype(int)
mode = [df["charged_off"].mode()[0],] * len(df)
print(roc_auc_score(df["charged_off"], mode))

0.5


In [20]:
months = {
    "Jan": 0,
    "Feb": 1,
    "Mar": 2,
    "Apr": 3,
    "May": 4,
    "Jun": 5,
    "Jul": 6,
    "Aug": 7,
    "Sep": 8,
    "Oct": 9,
    "Nov": 10,
    "Dec": 11
}
maps = {}

df = df.drop(columns=[
    "funded_amnt",
    "title"
])
df = df.dropna(axis=1, how='all')

df["term"] = df["term"].apply(lambda x: x[1:3]).astype(int)
df["int_rate"] = df["int_rate"].apply(lambda x: x[:-1]).astype(float)
df["zip_code"] = df["zip_code"].apply(lambda x: x[:-2]).astype(int)
df["earliest_cr_line"] = df["earliest_cr_line"].apply(lambda x: months[x.split("-")[0]]+int(x.split("-")[1])*12).astype(int)

df_train = df.sample(frac=0.8, random_state=1)
df_test  = df[~df.index.isin(df_train.index)]

for c in df_train:
    if df_train[c].dtype == "object":
        maps[c] = {}
        cats = list(df_train[c].astype('category').cat.categories.values)
        for cat in cats:
            maps[c][str(cat)] = random.uniform(0.0, 1.0)
        maps[c][None] = -1.0
        df_train[c] = df_train[c].apply(lambda x: maps[c].get(str(x), maps[c][None])).astype(float)

for c in df_test:
    if df_test[c].dtype == "object":
        df_test[c] = df_test[c].apply(lambda x: maps[c].get(str(x), maps[c][None])).astype(float)

col_rep_dict = {}        
cols = [
    "dti",
    "annual_inc_joint",
    "dti_joint",
    "il_util",
    "all_util",
    "avg_cur_bal",
    "bc_open_to_buy",
    "bc_util",
    "percent_bc_gt_75",
    "revol_bal_joint",
    "num_tl_120dpd_2m",
    "pct_tl_nvr_dlq",
    "sec_app_inq_last_6mths",
    "sec_app_mort_acc",
    "sec_app_open_acc",
    "sec_app_revol_util",
    "sec_app_open_act_il",
    "sec_app_num_rev_accts",
    "sec_app_chargeoff_within_12_mths",
    "sec_app_collections_12_mths_ex_med",
    "sec_app_mths_since_last_major_derog",
    "mths_since_last_delinq",
    "mths_since_last_record",
    "mths_since_last_major_derog",
    "mths_since_rcnt_il",
    "mo_sin_old_il_acct",
    "mths_since_recent_bc",
    "mths_since_recent_bc_dlq",
    "mths_since_recent_inq",
    "mths_since_recent_revol_delinq"
]
for c in cols:
    col_rep_dict[c] = df_train[c].min() - 1.0
    df_train[c] = df_train[c].fillna(col_rep_dict[c])
    
for c in cols:
    df_test[c] = df_test[c].fillna(col_rep_dict[c])

cols2 = [c for c in df_train if c is not "charged_off"]
means = df_train[cols2].mean()
stddevs = df_train[cols2].std()

df_train[cols2] = (df_train[cols2] - means) / stddevs
df_test[cols2] = (df_test[cols2] - means) / stddevs

C:\Users\brit2\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\brit2\Anaconda3\lib\site-packages\ipykernel_launcher.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\brit2\Anaconda3\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

In [5]:
df_train.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,...,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method,charged_off
id,,,,,,,,,,,,,,,,,,,,,
20726,-0.485614,-0.612655,0.290340,-0.334456,-0.940378,-0.416788,0.359776,0.848057,0.982154,0.777071,...,-0.304431,-0.328767,-0.331103,-0.289577,-0.316261,-0.340387,-0.358385,-0.180193,0.261878,1
35481,1.978719,1.632186,0.909443,1.541557,-1.153920,0.790286,0.786619,-0.079455,-1.051058,0.062580,...,-0.304431,-0.328767,-0.331103,-0.289577,-0.316261,-0.340387,-0.358385,-0.180193,0.261878,1
39942,-1.274200,-0.612655,0.114231,-1.283231,-0.940378,-0.839995,-2.639400,-2.678737,0.982154,-0.585177,...,-0.304431,-0.328767,-0.331103,-0.289577,-0.316261,-0.340387,-0.358385,-0.180193,0.261878,1
36509,1.978719,1.632186,-0.610174,1.022426,1.302805,-1.208181,0.166578,0.848057,0.982154,-0.092226,...,5.022045,3.540496,0.062241,2.129086,3.868742,2.269650,2.323597,-0.180193,0.261878,0
27939,-0.781334,-0.612655,-0.150839,-0.717142,-0.940378,-0.898686,0.293728,-0.369602,0.982154,0.181662,...,-0.304431,-0.328767,-0.331103,-0.289577,-0.316261,-0.340387,-0.358385,-0.180193,0.261878,1


In [6]:
df_test.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,...,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,disbursement_method,charged_off
id,,,,,,,,,,,,,,,,,,,,,
24414,-0.850335,-0.612655,0.114231,-0.781923,-0.940378,-0.839995,-2.639400,-0.807506,-1.051058,-0.485196,...,-0.304431,-0.328767,-0.331103,-0.289577,-0.316261,-0.340387,-0.358385,-0.180193,0.261878,0
16701,0.066397,-0.612655,-1.220199,0.118224,0.356726,-0.760889,0.209199,0.848057,0.982154,0.003039,...,-0.304431,-0.328767,-0.331103,-0.289577,-0.316261,-0.340387,-0.358385,-0.180193,0.261878,0
11504,1.978719,-0.612655,-1.476192,2.074829,0.356726,1.423062,-0.501797,0.848057,0.982154,1.312939,...,-0.304431,-0.328767,-0.331103,-0.289577,-0.316261,-0.340387,-0.358385,-0.180193,0.261878,0
19667,2.471585,-0.612655,-1.222015,2.674348,0.356726,-0.760889,-0.383202,-0.181982,0.982154,2.954399,...,-0.304431,-0.328767,-0.331103,-0.289577,-0.316261,-0.340387,-0.358385,-0.180193,0.261878,0
39343,0.302973,1.632186,-0.149024,-0.132652,-0.940378,-0.898686,0.397856,-0.369602,0.982154,0.741347,...,-0.304431,-0.328767,-0.331103,-0.289577,-0.316261,-0.340387,-0.358385,-0.180193,0.261878,0


In [7]:
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

In [21]:
print("XBG-Dart")
for i in range(1,10+1):
    model = XGBClassifier(booster='dart', max_depth=i, n_estimators=200, n_jobs=-1)
    model.fit(X_train, y_train)
    print(i, "\t", roc_auc_score(y_test, model.predict_proba(X_test)[:,1]))

XBG-Dart
1 	 0.7403551102925903
2 	 0.7531737679493749
3 	 0.7557349377570378
4 	 0.7608838639101596
5 	 0.7615935306091361
6 	 0.7572800033233172
7 	 0.7530719913593753
8 	 0.7481094479139261
9 	 0.7498771757342454
10 	 0.7477631305648254


In [8]:
X_train = df_train[[c for c in df_train if c != "charged_off"]]
y_train = df_train["charged_off"]

X_test = df_test[[c for c in df_test if c != "charged_off"]]
y_test = df_test["charged_off"]

print("Tree")
for i in range(1,10+1):
    model = DecisionTreeClassifier(max_depth=i)
    model.fit(X_train, y_train)
    print(i, "\t", roc_auc_score(y_test, model.predict_proba(X_test)[:,1]))

print("XBG")
for i in range(1,10+1):
    model = XGBClassifier(max_depth=i, n_estimators=200, n_jobs=-1)
    model.fit(X_train, y_train)
    print(i, "\t", roc_auc_score(y_test, model.predict_proba(X_test)[:,1]))
    
print("Forest")
for i in range(1,20+1):
    model = RandomForestClassifier(max_depth=i, n_estimators=200, n_jobs=-1, oob_score=True)
    model.fit(X_train, y_train)
    proba = model.oob_decision_function_[:, 1]
    print(i, "\t", roc_auc_score(y_train, proba), "\t", roc_auc_score(y_test, model.predict_proba(X_test)[:,1]))

Tree
1 	 0.6340139440851877
2 	 0.6730886771812732
3 	 0.6884341637010677
4 	 0.6864973621169531
5 	 0.7003119764044478
6 	 0.6881349959150892
7 	 0.6901280861846935
8 	 0.6846147721450629
9 	 0.6818212470747885
10 	 0.6727868784358253
XBG
1 	 0.7403551102925903
2 	 0.7531737679493749
3 	 0.7557349377570378
4 	 0.7608838639101596
5 	 0.7615935306091361
6 	 0.7572800033233172
7 	 0.7530719913593753
8 	 0.7481094479139261
9 	 0.7498771757342454
10 	 0.7477631305648254
Forest
1 	 0.6702920835181156 	 0.6807904648489969
2 	 0.7023859728001747 	 0.7094279740227369
3 	 0.7060489535179351 	 0.7130933159782323
4 	 0.713939590714766 	 0.7218245011562374
5 	 0.7202050876713131 	 0.7281455889887423
6 	 0.723746134028973 	 0.7293189969120843
7 	 0.7253433795284492 	 0.7356481160945485
8 	 0.7272623344428736 	 0.7375378373513162
9 	 0.7276228897374513 	 0.7372795879086642
10 	 0.7296073412229425 	 0.7377909633465803
11 	 0.7292781762726844 	 0.7396672528628994
12 	 0.7281936718537685 	 0.7400854369

In [9]:
import eli5
from eli5.sklearn import PermutationImportance

In [10]:
permuter = PermutationImportance(model, scoring='roc_auc', n_iter=5, cv='prefit')
permuter.fit(X_train, y_train)
eli5.show_weights(permuter, top=None, feature_names=X_train.columns.tolist())

Weight,Feature
0.0011 ± 0.0001,int_rate
0.0003 ± 0.0000,grade
0.0003 ± 0.0000,installment
0.0002 ± 0.0000,term
0.0002 ± 0.0000,dti
0.0002 ± 0.0000,tot_hi_cred_lim
0.0001 ± 0.0000,loan_amnt
0.0001 ± 0.0000,bc_open_to_buy
0.0001 ± 0.0000,mort_acc
0.0001 ± 0.0000,tot_cur_bal


In [22]:
X = pd.read_csv("./kaggle-dataset/test_features.csv", index_col='id')

X = X.drop(columns=[
    "funded_amnt",
    "title"
])
X = X.dropna(axis=1, how='all')

X["term"] = X["term"].apply(lambda x: x[1:3]).astype(int)
X["int_rate"] = X["int_rate"].apply(lambda x: x[:-1]).astype(float)
X["zip_code"] = X["zip_code"].apply(lambda x: x[:-2]).astype(int)
X["earliest_cr_line"] = X["earliest_cr_line"].apply(lambda x: months[x.split("-")[0]]+int(x.split("-")[1])*12).astype(int)

for c in X:
    if X[c].dtype == "object":
        X[c] = X[c].apply(lambda x: maps[c].get(str(x), maps[c][None])).astype(float)
for c in col_rep_dict:
    X[c] = X[c].fillna(col_rep_dict[c])

for c in X:
    if X[c].dtype == "object":
        X[c] = X[c].apply(lambda x: maps[c].get(str(x), maps[c][None])).astype(float)

for c in cols:
    X[c] = X[c].fillna(col_rep_dict[c])

X[cols2] = (X[cols2] - means) / stddevs

X["charged_off"] = XGBClassifier(booster='dart', n_estimators=200, max_depth=5, n_jobs=-1).fit(X_train, y_train).predict_proba(X)[:,1]
out = X[["charged_off"]]
out.to_csv("./kaggle-dataset/submission_3.csv", )

In [9]:
features = []
model = XGBClassifier(max_depth=7)
model.fit(X_train, y_train)
for i, c in enumerate(X_train):
    print("{:40s}{: .3%}".format(c, model.feature_importances_[i]))
    features.append({"feature": c, "value": model.feature_importances_[i]})

loan_amnt                                2.362%
term                                     0.255%
int_rate                                 3.626%
installment                              3.932%
grade                                    0.230%
sub_grade                                1.111%
emp_title                                2.349%
emp_length                               0.983%
home_ownership                           0.511%
annual_inc                               2.758%
purpose                                  0.996%
zip_code                                 3.038%
addr_state                               1.609%
dti                                      3.383%
delinq_2yrs                              0.051%
earliest_cr_line                         2.336%
inq_last_6mths                           0.766%
mths_since_last_delinq                   0.817%
mths_since_last_record                   0.511%
open_acc                                 0.600%
pub_rec                                 

In [10]:
feature_df = pd.DataFrame(features)
feature_df.index = feature_df["feature"]
feature_df = feature_df.drop(columns=["feature"])
feature_df.sort_values(by='value')

,value
feature,
acc_now_delinq,0.000000
num_tl_30dpd,0.000000
delinq_amnt,0.000000
chargeoff_within_12_mths,0.000128
sec_app_chargeoff_within_12_mths,0.000128
num_tl_90g_dpd_24m,0.000128
collections_12_mths_ex_med,0.000255
application_type,0.000255
pub_rec,0.000511


In [21]:
indexes = feature_df[feature_df["value"] > 0.001].index.values
X_train2 = X_train[indexes] 
X_test2 = X_test[indexes]

In [22]:
for i in range(1,10+1):
    probs = XGBClassifier(max_depth=i).fit(X_train2, y_train).predict_proba(X_test2)[:,1]
    print(i, "\t", roc_auc_score(y_test, probs))

1 	 0.73033537809657
2 	 0.7469694808701551
3 	 0.7525669856128058
4 	 0.7563482282565047
5 	 0.7578872703103148
6 	 0.76056281761912
7 	 0.7579502056302534
8 	 0.7516541119127076
9 	 0.7492120968746971
10 	 0.7527822396388661


In [23]:
X = pd.read_csv("./kaggle-dataset/test_features.csv", index_col='id')

X = X.drop(columns=[
    "funded_amnt",
    "title"
])
X = X.dropna(axis=1, how='all')

X["term"] = X["term"].apply(lambda x: x[1:3]).astype(int)
X["int_rate"] = X["int_rate"].apply(lambda x: x[:-1]).astype(float)
X["zip_code"] = X["zip_code"].apply(lambda x: x[:-2]).astype(int)
X["earliest_cr_line"] = X["earliest_cr_line"].apply(lambda x: months[x.split("-")[0]]+int(x.split("-")[1])*12).astype(int)

for c in X:
    if X[c].dtype == "object":
        X[c] = X[c].apply(lambda x: maps[c].get(str(x), maps[c][None])).astype(float)
for c in col_rep_dict:
    X[c] = X[c].fillna(col_rep_dict[c])

X["charged_off"] = XGBClassifier(max_depth=6).fit(X_train2, y_train).predict_proba(X[indexes])[:,1]
out = X[["charged_off"]]
out.to_csv("./kaggle-dataset/submission_2.csv", )